In [6]:
import requests
import datetime
import time
import csv
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')
import xgboost as xgb
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
import seaborn as sns

In [35]:
models_df = pd.read_csv('models_df.csv')
models_df.drop('Unnamed: 0', axis=1, inplace=True)
models_df

,fight,loser,winner,date,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_w,implied_prob_l,prev_w,prev_l,bday_w,height_w,weight_w,reach_w,stance_w,bday_l,height_l,weight_l,reach_l,stance_l,age_w,age_l,weightclass,location
0,Charles Oliveira vs Jared Gordon,Jared-Gordon,Charles-Oliveira,2019-11-16,77.933333,950.0,1030.0,55.0,44.0,0.004031,0.000000,1.507616,0.371220,1.855883,0.354357,0.011210,0.011931,0.148884,0.109668,0.055605,0.113352,0.004031,0.937505,0.370354,0.199757,0.952281,0.271547,0.283787,15,8,26.016667,557.0,572.0,16.0,18.0,0.019544,0.000000,2.268233,0.398209,1.970380,0.215258,0.004388,0.004860,0.125000,0.129960,0.022023,0.110399,0.010982,1.689653,0.375148,0.203431,1.354574,0.201881,0.711778,3,2,0.793651,0.212766,23,5,1989-10-17,70.0,155.0,74.0,Orthodox,1988-09-06,69.0,155.0,68.0,Orthodox,30.0,31.0,Lightweight,"Sao Paulo, Sao Paulo, Brazil"
1,Charles Oliveira vs Jared Gordon,Jared-Gordon,Charles-Oliveira,2019-11-16,77.933333,950.0,1030.0,55.0,44.0,0.004031,0.000000,1.507616,0.371220,1.855883,0.354357,0.011210,0.011931,0.148884,0.109668,0.055605,0.113352,0.004031,0.937505,0.370354,0.199757,0.952281,0.271547,0.283787,15,8,26.016667,557.0,572.0,16.0,18.0,0.019544,0.000000,2.268233,0.398209,1.970380,0.215258,0.004388,0.004860,0.125000,0.129960,0.022023,0.110399,0.010982,1.689653,0.375148,0.203431,1.354574,0.201881,0.711778,3,2,0.793651,0.212766,23,5,1989-10-17,70.0,155.0,74.0,Orthodox,1988-09-06,69.0,155.0,68.0,Orthodox,30.0,31.0,Lightweight,"Sao Paulo, Sao Paulo, Brazil"
2,Charles Oliveira vs Jared Gordon,Jared-Gordon,Charles-Oliveira,2019-11-16,77.933333,950.0,1030.0,55.0,44.0,0.004031,0.000000,1.507616,0.371220,1.855883,0.354357,0.011210,0.011931,0.148884,0.109668,0.055605,0.113352,0.004031,0.937505,0.370354,0.199757,0.952281,0.271547,0.283787,15,8,26.016667,557.0,572.0,16.0,18.0,0.019544,0.000000,2.268233,0.398209,1.970380,0.215258,0.004388,0.004860,0.125000,0.129960,0.022023,0.110399,0.010982,1.689653,0.375148,0.203431,1.354574,0.201881,0.711778,3,2,0.793651,0.212766,23,5,1989-10-17,70.0,155.0,74.0,Orthodox,1988-09-06,69.0,155.0,68.0,Orthodox,30.0,31.0,Lightweight,"Sao Paulo, Sao Paulo, Brazil"
3,Charles Oliveira vs Jared Gordon,Jared-Gordon,Charles-Oliveira,2019-11-16,77.933333,950.0,1030.0,55.0,44.0,0.004031,0.000000,1.507616,0.371220,1.855883,0.354357,0.011210,0.011931,0.148884,0.109668,0.055605,0.113352,0.004031,0.937505,0.370354,0.199757,0.952281,0.271547,0.283787,15,8,26.016667,557.0,572.0,16.0,18.0,0.019544,0.000000,2.268233,0.398209,1.970380,0.215258,0.004388,0.004860,0.125000,0.129960,0.022023,0.110399,0.010982,1.689653,0.375148,0.203431,1.354574,0.201881,0.711778,3,2,0.793651,0.212766,23,5,1989-10-17,70.0,155.0,74.0,Orthodox,1988-09-06,69.0,155.0,68.0,Orthodox,30.0,31.0,Lightweight,"Sao Paulo, Sao Paulo, Brazil"
4,Joaquim Silva vs Jared Gordon,Jared-Gordon,Joaquim-Silva,2018-12-15,21.566667,332.0,497.0,14.0,7.0,0.081826,0.000000,1.285831,0.243191,1.210240,0.203499,0.005789,0.004788,0.027473,0.202381,0.000000,0.013248,0.000000,1.040744,0.205343,0.039744,0.790591,0.072864,0.422376,3,1,14.366667,287.0,305.0,9.0,8.0,0.035393,0.000000,2.208209,0.565847,1.493689,0.255516,0.004665,0.006044,0.074074,0.214286,0.009944,0.135235,0.000000,1.823902,0.190515,0.193792,1.116386,0.061503,1.030320,2,1,0.487805,0.529101,4,3,1989-02-05,68.0,155.0,69.0,Southpaw,1988-09-06,69.0,155.0,68.0,Orthodox,29.0,30.0,Lightweight,"Milwaukee, Wisconsin, US

In [36]:
# country = []
# for i in list(models_df['location']):
#     country.append(i[-5:])

# models_df['country'] = country

In [37]:
newest = pd.get_dummies(data=models_df, columns=['stance_w', 'stance_l', 'weightclass'])

In [38]:
newest = newest.sample(frac=1)

In [39]:
newest

,fight,loser,winner,date,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_w,implied_prob_l,prev_w,prev_l,bday_w,height_w,weight_w,reach_w,bday_l,height_l,weight_l,reach_l,age_w,age_l,location,stance_w_Orthodox,stance_w_Southpaw,stance_w_Switch,stance_l_Orthodox,stance_l_Southpaw,stance_l_Switch,weightclass_Bantamweight,weightclass_Catch Weight,weightclass_Featherweight,weightclass_Flyweight,weightclass_Heavyweight,weightclass_Light Heavyweight,weightclass_Lightweight,weightclass_Middleweight,weightclass_Welterweight,weightclass_Women's Bantamweight,weightclass_Women's Featherweight,weightclass_Women's Flyweight,weightclass_Women's Strawweight
938,Alistair Overeem vs Roy Nelson,Roy-Nelson,Alistair-Overeem,2015-03-14,22.516667,168.0,261.0,7.0,3.0,0.048292,0.022206,1.169837,0.464874,2.522379,1.353868,0.016247,0.006431,0.119048,0.666667,0.006345,0.006345,0.000000,0.885447,0.216821,0.067569,0.639187,0.247477,0.283172,3,3,56.383333,991.0,806.0,22.0,23.0,0.070044,0.000000,1.704282,0.396245,1.213001,0.308447,0.005746,0.009133,0.058712,0.115217,0.000000,0.016427,0.000000,1.350594,0.227360,0.126328,1.239771,0.258435,0.206076,7,6,0.649351,0.357143,6,13,1980-05-17,76.0,265.0,80.0,1976-06-20,72.0,263.0,72.0,34.0,38.0,"Dallas, Texas, USA",1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
417,Edson Barboza vs Kevin Lee,Edson-Barboza,Kevin-Lee,2018-04-21,65.833333,827.0,1016.0,44.0,45.0,0.000000,0.000000,0.883885,0.147262,1.225874,0.368893,0.006146,0.005955,0.065431,0.123272,0.025152,0.044842,0.000000,0.561697,0.222369,0.099819,0.636277,0.109170,0.138438,9,3,93.050000,1862.0,1869.0,35.0,44.0,0.019665,0.000000,1.584809,0.248992,1.493655,0.225384,0.004075,0.005624,0.117143,0.243007,0.008707,0.010538,0.000000,0.847731,0.341016,0.396063,1.407833,0.029559,0.147418,13,5,0.591716,0.392157,12,18,1992-09-04,69.0,170.0,77.0,1986-01-21,71.0,155.0,75.0,25.0,32.0,"Atlantic City, New Jersey, USA",1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1289,Johny Hendricks vs Stephen Thompson,Johny-Hendricks,Stephen-Thompson,2016-02-06,35.866667,535.0,524.0,13.0,19.0,0.006219,0.013941,1.156838,0.158998,1.437994,0.354350,0.005278,0.006445,0.038462,0.120108,0.000000,0.003983,0.003983,0.518839,0.272295,0.365703,0.943519,0.201370,0.011949,6,1,80.700000,1606.0,1341.0,53.0,80.0,0.064497,0.000000,1.579997,0.291542,1.185315,0.116347,0.004438,0.006240,0.059748,0.071179,0.008851,0.023013,0.000000,1.093823,0.307067,0.179107,0.828193,0.493205,0.258599,12,3,0.299401,0.704225,7,15,1983-02-11,72.0,170.0,75.0,1983-09-12,69.0,185.0,69.0,32.0,32.0,"Las Vegas, Nevada, USA",1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
615,Jim Miller vs Danny Castillo,Danny-Castillo,Jim-Miller,2015-07-25,91.600000,1379.0,1013.0,63.0,29.0,0.012972,0.000000,1.129775,0.396713,0.693205,0.201412,0.004865,0.011578,0.138473,0.265517,0.064909,0.080541,0.003119,0.718582,0.208417,0.202776,0.631183,0.207402,0.291190,13,5,67.466667,823.0,752.0,39.0,62.0,0.024860,0.014822,0.994368,0.306697,0.957205,0.326799,0.006241,0.008421,0.074786,0.083529,0.032468,0.048127,0.017332,0.484676,0.180873,0.328819,0.622347,0.164534,0.207487,7,5,0.534759,0.476190,18,12,1983-08-30,68.0,155.0,71.0,1979-08-25,69.0,155.0,71.0,31.0,35.0,"Chicago, Illinois, USA",0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
969,Brad Tavares vs Thales Leites,Thales-Leites,Brad-Tavares,2017-10-07,86.366667,1174.0,1118.0,45.0,65.0,0.018461,0.011579,1.014435,0.263646,0.849236,0.146427,0.005841,0.007274,0.093704,0

In [40]:
newest.columns

Index(['fight', 'loser', 'winner', 'date', 'cumultime_w',
       'cumulStr_Att_fighter_w', 'cumulStr_Att_oponent_w',
       'cumulTD_Att_fighter_w', 'cumulTD_Att_oponent_w', 'KDPM_w',
       'KDPM_ratio_w', 'SLPM_w', 'SLPM_ratio_w', 'SAPM_w', 'SAPM_ratio_w',
       'Str_Acc_w', 'Str_Def_w', 'TD_Acc_w', 'TD_Def_w', 'SubPM_w', 'PassPM_w',
       'RevPM_w', 'Head_SLPM_w', 'Body_SLPM_w', 'Leg_SLPM_w', 'Dist_SLPM_w',
       'Clinch_SLPM_w', 'Ground_SLPM_w', 'wins_w', 'losses_w', 'cumultime_l',
       'cumulStr_Att_fighter_l', 'cumulStr_Att_oponent_l',
       'cumulTD_Att_fighter_l', 'cumulTD_Att_oponent_l', 'KDPM_l',
       'KDPM_ratio_l', 'SLPM_l', 'SLPM_ratio_l', 'SAPM_l', 'SAPM_ratio_l',
       'Str_Acc_l', 'Str_Def_l', 'TD_Acc_l', 'TD_Def_l', 'SubPM_l', 'PassPM_l',
       'RevPM_l', 'Head_SLPM_l', 'Body_SLPM_l', 'Leg_SLPM_l', 'Dist_SLPM_l',
       'Clinch_SLPM_l', 'Ground_SLPM_l', 'wins_l', 'losses_l',
       'implied_prob_w', 'implied_prob_l', 'prev_w', 'prev_l', 'bday_w',
       'heig

In [41]:
winners = newest[['fight', 'date', 'winner', 'cumultime_w',
       'cumulStr_Att_fighter_w', 'cumulStr_Att_oponent_w',
       'cumulTD_Att_fighter_w', 'cumulTD_Att_oponent_w', 'KDPM_w',
       'KDPM_ratio_w', 'SLPM_w', 'SLPM_ratio_w', 'SAPM_w', 'SAPM_ratio_w',
       'Str_Acc_w', 'Str_Def_w', 'TD_Acc_w', 'TD_Def_w', 'SubPM_w', 'PassPM_w',
       'RevPM_w', 'Head_SLPM_w', 'Body_SLPM_w', 'Leg_SLPM_w', 'Dist_SLPM_w',
       'Clinch_SLPM_w', 'Ground_SLPM_w', 'wins_w', 'losses_w', 'implied_prob_w',
       'prev_w', 'height_w', 'weight_w', 'reach_w', 'age_w', 'stance_w_Orthodox',
       'stance_w_Southpaw', 'stance_w_Switch']]

In [42]:
losers = newest[['fight', 'date', 'loser', 'cumultime_l',
       'cumulStr_Att_fighter_l', 'cumulStr_Att_oponent_l',
       'cumulTD_Att_fighter_l', 'cumulTD_Att_oponent_l', 'KDPM_l',
       'KDPM_ratio_l', 'SLPM_l', 'SLPM_ratio_l', 'SAPM_l', 'SAPM_ratio_l',
       'Str_Acc_l', 'Str_Def_l', 'TD_Acc_l', 'TD_Def_l', 'SubPM_l', 'PassPM_l',
       'RevPM_l', 'Head_SLPM_l', 'Body_SLPM_l', 'Leg_SLPM_l', 'Dist_SLPM_l',
       'Clinch_SLPM_l', 'Ground_SLPM_l', 'wins_l', 'losses_l', 'implied_prob_l',
       'prev_l', 'height_l', 'weight_l', 'reach_l', 'age_l', 'stance_l_Orthodox',
       'stance_l_Southpaw', 'stance_l_Switch']]

In [43]:
len(winners)

1454

In [44]:
len(losers)

1454

In [45]:
winners.rename(columns={'winner': 'fighter'}, inplace=True)
losers.rename(columns={'loser': 'fighter'}, inplace=True)

In [46]:
winners

,fight,date,fighter,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,implied_prob_w,prev_w,height_w,weight_w,reach_w,age_w,stance_w_Orthodox,stance_w_Southpaw,stance_w_Switch
938,Alistair Overeem vs Roy Nelson,2015-03-14,Alistair-Overeem,22.516667,168.0,261.0,7.0,3.0,0.048292,0.022206,1.169837,0.464874,2.522379,1.353868,0.016247,0.006431,0.119048,0.666667,0.006345,0.006345,0.000000,0.885447,0.216821,0.067569,0.639187,0.247477,0.283172,3,3,0.649351,6,76.0,265.0,80.0,34.0,1,0,0
417,Edson Barboza vs Kevin Lee,2018-04-21,Kevin-Lee,65.833333,827.0,1016.0,44.0,45.0,0.000000,0.000000,0.883885,0.147262,1.225874,0.368893,0.006146,0.005955,0.065431,0.123272,0.025152,0.044842,0.000000,0.561697,0.222369,0.099819,0.636277,0.109170,0.138438,9,3,0.591716,12,69.0,170.0,77.0,25.0,1,0,0
1289,Johny Hendricks vs Stephen Thompson,2016-02-06,Stephen-Thompson,35.866667,535.0,524.0,13.0,19.0,0.006219,0.013941,1.156838,0.158998,1.437994,0.354350,0.005278,0.006445,0.038462,0.120108,0.000000,0.003983,0.003983,0.518839,0.272295,0.365703,0.943519,0.201370,0.011949,6,1,0.299401,7,72.0,170.0,75.0,32.0,1,0,0
615,Jim Miller vs Danny Castillo,2015-07-25,Jim-Miller,91.600000,1379.0,1013.0,63.0,29.0,0.012972,0.000000,1.129775,0.396713,0.693205,0.201412,0.004865,0.011578,0.138473,0.265517,0.064909,0.080541,0.003119,0.718582,0.208417,0.202776,0.631183,0.207402,0.291190,13,5,0.534759,18,68.0,155.0,71.0,31.0,0,1,0
969,Brad Tavares vs Thales Leites,2017-10-07,Brad-Tavares,86.366667,1174.0,1118.0,45.0,65.0,0.018461,0.011579,1.014435,0.263646,0.849236,0.146427,0.005841,0.007274,0.093704,0.148816,0.005980,0.019213,0.004962,0.576075,0.178404,0.259956,0.660712,0.251263,0.102460,10,4,0.628931,14,73.0,185.0,74.0,29.0,1,0,0
470,Tony Ferguson vs Gleison Tibau,2015-02-28,Tony-Ferguson,41.183333,771.0,646.0,6.0,22.0,0.009391,0.000000,1.130481,0.280324,0.826973,0.161306,0.003592,0.008128,0.416667,0.212879,0.030435,0.010489,0.003469,0.733841,0.190889,0.205751,1.061484,0.047953,0.021044,7,1,0.699301,8,71.0,155.0,76.0,31.0,1,0,0
1356,Yves Jabouin vs Felipe Arantes,2015-08-23,Felipe-Arantes,38.533333,517.0,489.0,13.0,25.0,0.003707,0.000000,0.826463,0.174372,0.817439,0.277643,0.004943,0.006641,0.112179,0.092000,0.007415,0.033226,0.018467,0.503922,0.237271,0.085269,0.559742,0.140880,0.125841,3,3,0.675676,6,68.0,135.0,73.0,27.0,1,0,0
533,Gray Maynard vs Nik Lentz,2018-10-06,Nik-Lentz,109.683333,1392.0,1399.0,66.0,153.0,0.005484,0.000000,1.167376,0.237968,1.143597,0.216191,0.007020,0.006076,0.106041,0.077417,0.029445,0.051237,0.004259,0.767369,0.253134,0.146873,0.637621,0.276917,0.252838,12,6,0.689655,18,68.0,155.0,68.0,34.0,1,0,0
243,Tatiana Suarez vs Carla Esparza,2018-09-08,Tatiana-Suarez,13.450000,133.0,36.0,8.0,3.0,0.000000,0.000000,1.237391,1.818702,0.212484,0.043043,0.017007,0.054516,0.225000,0.111111,0.020004,0.138078,0.000000,0.886830,0.223105,0.127456,0.136839,0.140668,0.959884,3,0,0.800000,3,65.0,115.0,66.0,27.0,0,0,0
646,Jose Aldo vs Max Holloway,2017-06-03,Max-Holloway,90.716667,2408.0,2159.0,16.0,84.0,0.009578,0.000000,1.869720,0.264903,1.446777,0.175281,0.002744,0.004715,0.234375,0.119907,0.007052,0.021835,0.000000,1.305720,0.394927,0.169073,1.441886,0.294687,0.133147,13,3,0.434783,16,71.0,145.0,69.0,25.0,1,0,0


In [47]:
losers

,fight,date,fighter,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_l,prev_l,height_l,weight_l,reach_l,age_l,stance_l_Orthodox,stance_l_Southpaw,stance_l_Switch
938,Alistair Overeem vs Roy Nelson,2015-03-14,Roy-Nelson,56.383333,991.0,806.0,22.0,23.0,0.070044,0.000000,1.704282,0.396245,1.213001,0.308447,0.005746,0.009133,0.058712,0.115217,0.000000,0.016427,0.000000,1.350594,0.227360,0.126328,1.239771,0.258435,0.206076,7,6,0.357143,13,72.0,263.0,72.0,38.0,1,0,0
417,Edson Barboza vs Kevin Lee,2018-04-21,Edson-Barboza,93.050000,1862.0,1869.0,35.0,44.0,0.019665,0.000000,1.584809,0.248992,1.493655,0.225384,0.004075,0.005624,0.117143,0.243007,0.008707,0.010538,0.000000,0.847731,0.341016,0.396063,1.407833,0.029559,0.147418,13,5,0.392157,18,71.0,155.0,75.0,32.0,1,0,0
1289,Johny Hendricks vs Stephen Thompson,2016-02-06,Johny-Hendricks,80.700000,1606.0,1341.0,53.0,80.0,0.064497,0.000000,1.579997,0.291542,1.185315,0.116347,0.004438,0.006240,0.059748,0.071179,0.008851,0.023013,0.000000,1.093823,0.307067,0.179107,0.828193,0.493205,0.258599,12,3,0.704225,15,69.0,185.0,69.0,32.0,0,1,0
615,Jim Miller vs Danny Castillo,2015-07-25,Danny-Castillo,67.466667,823.0,752.0,39.0,62.0,0.024860,0.014822,0.994368,0.306697,0.957205,0.326799,0.006241,0.008421,0.074786,0.083529,0.032468,0.048127,0.017332,0.484676,0.180873,0.328819,0.622347,0.164534,0.207487,7,5,0.476190,12,69.0,155.0,71.0,35.0,1,0,0
969,Brad Tavares vs Thales Leites,2017-10-07,Thales-Leites,101.716667,1200.0,1329.0,102.0,24.0,0.015917,0.009831,0.869445,0.456280,0.974502,0.287627,0.006183,0.007738,0.050811,0.116071,0.034083,0.089994,0.001404,0.616812,0.069419,0.183214,0.539190,0.100165,0.230090,12,6,0.384615,18,73.0,185.0,78.0,36.0,1,0,0
470,Tony Ferguson vs Gleison Tibau,2015-02-28,Gleison-Tibau,140.633333,1806.0,1763.0,156.0,69.0,0.004399,0.000000,0.730136,0.227649,0.716796,0.198496,0.004778,0.008978,0.069253,0.216069,0.023542,0.069024,0.003809,0.546610,0.116536,0.066990,0.486431,0.119350,0.124356,16,8,0.309598,24,70.0,155.0,71.0,31.0,0,1,0
1356,Yves Jabouin vs Felipe Arantes,2015-08-23,Yves-Jabouin,52.750000,812.0,662.0,17.0,25.0,0.002708,0.000000,1.078558,0.260254,0.803939,0.129657,0.004967,0.008022,0.115196,0.229524,0.009614,0.024374,0.017738,0.582313,0.308166,0.188078,0.826773,0.115296,0.136489,5,4,0.327869,9,67.0,135.0,68.0,36.0,1,0,0
533,Gray Maynard vs Nik Lentz,2018-10-06,Gray-Maynard,100.383333,1414.0,1525.0,64.0,69.0,0.088677,0.009962,1.095625,0.293181,0.662905,0.135477,0.004913,0.007102,0.093477,0.118585,0.013915,0.048386,0.004269,0.855469,0.189076,0.051080,0.590730,0.237212,0.267683,11,6,0.317460,17,69.0,155.0,70.0,39.0,1,0,0
243,Tatiana Suarez vs Carla Esparza,2018-09-08,Carla-Esparza,43.716667,657.0,590.0,35.0,47.0,0.000000,0.000000,1.153048,0.476364,0.759393,0.117923,0.004950,0.007642,0.076871,0.089056,0.006662,0.062850,0.003268,0.985711,0.131391,0.035946,0.545149,0.207917,0.399982,4,3,0.222222,7,61.0,115.0,63.0,30.0,0,0,0
646,Jose Aldo vs Max Holloway,2017-06-03,Jose-Aldo,64.200000,1279.0,1328.0,17.0,64.0,0.078478,0.000000,1.298301,0.240555,0.802383,0.103518,0.003266,0.004559,0.140523,0.093342,0.005192,0.021807,0.000000,0.990291,0.108006,0.200004,1.025714,0.030817,0.241770,8,1,0.584795,9,67.0,145.0,70.0,30.0,1,0,0


In [48]:
winners.columns = [c.replace('_w', '') for c in winners]
losers.columns = [c.replace('_l', '') for c in losers]

In [49]:
fighter1 = pd.DataFrame()
fighter1 = fighter1.append(winners[0:len(fighter1)//2])
fighter1 = fighter1.append(losers[len(fighter1)//2:])

In [50]:
fighter1

,fight,date,fighter,cumultime,cumulStr_Att_fighter,cumulStr_Att_oponent,cumulTD_Att_fighter,cumulTD_Att_oponent,KDPM,KDPM_ratio,SLPM,SLPM_ratio,SAPM,SAPM_ratio,Str_Acc,Str_Def,TD_Acc,TD_Def,SubPM,PassPM,RevPM,Head_SLPM,Body_SLPM,Leg_SLPM,Dist_SLPM,Clinch_SLPM,Ground_SLPM,wins,losses,implied_prob,prev,height,weight,reach,age,stance_Orthodox,stance_Southpaw,stance_Switch
938,Alistair Overeem vs Roy Nelson,2015-03-14,Roy-Nelson,56.383333,991.0,806.0,22.0,23.0,0.070044,0.000000,1.704282,0.396245,1.213001,0.308447,0.005746,0.009133,0.058712,0.115217,0.000000,0.016427,0.000000,1.350594,0.227360,0.126328,1.239771,0.258435,0.206076,7,6,0.357143,13,72.0,263.0,72.0,38.0,1,0,0
417,Edson Barboza vs Kevin Lee,2018-04-21,Edson-Barboza,93.050000,1862.0,1869.0,35.0,44.0,0.019665,0.000000,1.584809,0.248992,1.493655,0.225384,0.004075,0.005624,0.117143,0.243007,0.008707,0.010538,0.000000,0.847731,0.341016,0.396063,1.407833,0.029559,0.147418,13,5,0.392157,18,71.0,155.0,75.0,32.0,1,0,0
1289,Johny Hendricks vs Stephen Thompson,2016-02-06,Johny-Hendricks,80.700000,1606.0,1341.0,53.0,80.0,0.064497,0.000000,1.579997,0.291542,1.185315,0.116347,0.004438,0.006240,0.059748,0.071179,0.008851,0.023013,0.000000,1.093823,0.307067,0.179107,0.828193,0.493205,0.258599,12,3,0.704225,15,69.0,185.0,69.0,32.0,0,1,0
615,Jim Miller vs Danny Castillo,2015-07-25,Danny-Castillo,67.466667,823.0,752.0,39.0,62.0,0.024860,0.014822,0.994368,0.306697,0.957205,0.326799,0.006241,0.008421,0.074786,0.083529,0.032468,0.048127,0.017332,0.484676,0.180873,0.328819,0.622347,0.164534,0.207487,7,5,0.476190,12,69.0,155.0,71.0,35.0,1,0,0
969,Brad Tavares vs Thales Leites,2017-10-07,Thales-Leites,101.716667,1200.0,1329.0,102.0,24.0,0.015917,0.009831,0.869445,0.456280,0.974502,0.287627,0.006183,0.007738,0.050811,0.116071,0.034083,0.089994,0.001404,0.616812,0.069419,0.183214,0.539190,0.100165,0.230090,12,6,0.384615,18,73.0,185.0,78.0,36.0,1,0,0
470,Tony Ferguson vs Gleison Tibau,2015-02-28,Gleison-Tibau,140.633333,1806.0,1763.0,156.0,69.0,0.004399,0.000000,0.730136,0.227649,0.716796,0.198496,0.004778,0.008978,0.069253,0.216069,0.023542,0.069024,0.003809,0.546610,0.116536,0.066990,0.486431,0.119350,0.124356,16,8,0.309598,24,70.0,155.0,71.0,31.0,0,1,0
1356,Yves Jabouin vs Felipe Arantes,2015-08-23,Yves-Jabouin,52.750000,812.0,662.0,17.0,25.0,0.002708,0.000000,1.078558,0.260254,0.803939,0.129657,0.004967,0.008022,0.115196,0.229524,0.009614,0.024374,0.017738,0.582313,0.308166,0.188078,0.826773,0.115296,0.136489,5,4,0.327869,9,67.0,135.0,68.0,36.0,1,0,0
533,Gray Maynard vs Nik Lentz,2018-10-06,Gray-Maynard,100.383333,1414.0,1525.0,64.0,69.0,0.088677,0.009962,1.095625,0.293181,0.662905,0.135477,0.004913,0.007102,0.093477,0.118585,0.013915,0.048386,0.004269,0.855469,0.189076,0.051080,0.590730,0.237212,0.267683,11,6,0.317460,17,69.0,155.0,70.0,39.0,1,0,0
243,Tatiana Suarez vs Carla Esparza,2018-09-08,Carla-Esparza,43.716667,657.0,590.0,35.0,47.0,0.000000,0.000000,1.153048,0.476364,0.759393,0.117923,0.004950,0.007642,0.076871,0.089056,0.006662,0.062850,0.003268,0.985711,0.131391,0.035946,0.545149,0.207917,0.399982,4,3,0.222222,7,61.0,115.0,63.0,30.0,0,0,0
646,Jose Aldo vs Max Holloway,2017-06-03,Jose-Aldo,64.200000,1279.0,1328.0,17.0,64.0,0.078478,0.000000,1.298301,0.240555,0.802383,0.103518,0.003266,0.004559,0.140523,0.093342,0.005192,0.021807,0.000000,0.990291,0.108006,0.200004,1.025714,0.030817,0.241770,8,1,0.584795,9,67.0,145.0,70.0,30.0,1,0,0


In [51]:
fighter2 = pd.DataFrame()
fighter2 = fighter2.append(losers[0:len(fighter2)//2])
fighter2 = fighter2.append(winners[len(fighter2)//2:])

In [52]:
fighter2

,fight,date,fighter,cumultime,cumulStr_Att_fighter,cumulStr_Att_oponent,cumulTD_Att_fighter,cumulTD_Att_oponent,KDPM,KDPM_ratio,SLPM,SLPM_ratio,SAPM,SAPM_ratio,Str_Acc,Str_Def,TD_Acc,TD_Def,SubPM,PassPM,RevPM,Head_SLPM,Body_SLPM,Leg_SLPM,Dist_SLPM,Clinch_SLPM,Ground_SLPM,wins,losses,implied_prob,prev,height,weight,reach,age,stance_Orthodox,stance_Southpaw,stance_Switch
938,Alistair Overeem vs Roy Nelson,2015-03-14,Alistair-Overeem,22.516667,168.0,261.0,7.0,3.0,0.048292,0.022206,1.169837,0.464874,2.522379,1.353868,0.016247,0.006431,0.119048,0.666667,0.006345,0.006345,0.000000,0.885447,0.216821,0.067569,0.639187,0.247477,0.283172,3,3,0.649351,6,76.0,265.0,80.0,34.0,1,0,0
417,Edson Barboza vs Kevin Lee,2018-04-21,Kevin-Lee,65.833333,827.0,1016.0,44.0,45.0,0.000000,0.000000,0.883885,0.147262,1.225874,0.368893,0.006146,0.005955,0.065431,0.123272,0.025152,0.044842,0.000000,0.561697,0.222369,0.099819,0.636277,0.109170,0.138438,9,3,0.591716,12,69.0,170.0,77.0,25.0,1,0,0
1289,Johny Hendricks vs Stephen Thompson,2016-02-06,Stephen-Thompson,35.866667,535.0,524.0,13.0,19.0,0.006219,0.013941,1.156838,0.158998,1.437994,0.354350,0.005278,0.006445,0.038462,0.120108,0.000000,0.003983,0.003983,0.518839,0.272295,0.365703,0.943519,0.201370,0.011949,6,1,0.299401,7,72.0,170.0,75.0,32.0,1,0,0
615,Jim Miller vs Danny Castillo,2015-07-25,Jim-Miller,91.600000,1379.0,1013.0,63.0,29.0,0.012972,0.000000,1.129775,0.396713,0.693205,0.201412,0.004865,0.011578,0.138473,0.265517,0.064909,0.080541,0.003119,0.718582,0.208417,0.202776,0.631183,0.207402,0.291190,13,5,0.534759,18,68.0,155.0,71.0,31.0,0,1,0
969,Brad Tavares vs Thales Leites,2017-10-07,Brad-Tavares,86.366667,1174.0,1118.0,45.0,65.0,0.018461,0.011579,1.014435,0.263646,0.849236,0.146427,0.005841,0.007274,0.093704,0.148816,0.005980,0.019213,0.004962,0.576075,0.178404,0.259956,0.660712,0.251263,0.102460,10,4,0.628931,14,73.0,185.0,74.0,29.0,1,0,0
470,Tony Ferguson vs Gleison Tibau,2015-02-28,Tony-Ferguson,41.183333,771.0,646.0,6.0,22.0,0.009391,0.000000,1.130481,0.280324,0.826973,0.161306,0.003592,0.008128,0.416667,0.212879,0.030435,0.010489,0.003469,0.733841,0.190889,0.205751,1.061484,0.047953,0.021044,7,1,0.699301,8,71.0,155.0,76.0,31.0,1,0,0
1356,Yves Jabouin vs Felipe Arantes,2015-08-23,Felipe-Arantes,38.533333,517.0,489.0,13.0,25.0,0.003707,0.000000,0.826463,0.174372,0.817439,0.277643,0.004943,0.006641,0.112179,0.092000,0.007415,0.033226,0.018467,0.503922,0.237271,0.085269,0.559742,0.140880,0.125841,3,3,0.675676,6,68.0,135.0,73.0,27.0,1,0,0
533,Gray Maynard vs Nik Lentz,2018-10-06,Nik-Lentz,109.683333,1392.0,1399.0,66.0,153.0,0.005484,0.000000,1.167376,0.237968,1.143597,0.216191,0.007020,0.006076,0.106041,0.077417,0.029445,0.051237,0.004259,0.767369,0.253134,0.146873,0.637621,0.276917,0.252838,12,6,0.689655,18,68.0,155.0,68.0,34.0,1,0,0
243,Tatiana Suarez vs Carla Esparza,2018-09-08,Tatiana-Suarez,13.450000,133.0,36.0,8.0,3.0,0.000000,0.000000,1.237391,1.818702,0.212484,0.043043,0.017007,0.054516,0.225000,0.111111,0.020004,0.138078,0.000000,0.886830,0.223105,0.127456,0.136839,0.140668,0.959884,3,0,0.800000,3,65.0,115.0,66.0,27.0,0,0,0
646,Jose Aldo vs Max Holloway,2017-06-03,Max-Holloway,90.716667,2408.0,2159.0,16.0,84.0,0.009578,0.000000,1.869720,0.264903,1.446777,0.175281,0.002744,0.004715,0.234375,0.119907,0.007052,0.021835,0.000000,1.305720,0.394927,0.169073,1.441886,0.294687,0.133147,13,3,0.434783,16,71.0,145.0,69.0,25.0,1,0,0


In [53]:
fighter1 = fighter1.add_suffix('_1')
fighter2 = fighter2.add_suffix('_2')

In [54]:
fighter1.rename(columns={'date_1': 'date'}, inplace=True)
fighter2.rename(columns={'date_2': 'date'}, inplace=True)

fighter1.rename(columns={'fight_1': 'fight'}, inplace=True)
fighter2.rename(columns={'fight_2': 'fight'}, inplace=True)

In [55]:
fighter1

,fight,date,fighter_1,cumultime_1,cumulStr_Att_fighter_1,cumulStr_Att_oponent_1,cumulTD_Att_fighter_1,cumulTD_Att_oponent_1,KDPM_1,KDPM_ratio_1,SLPM_1,SLPM_ratio_1,SAPM_1,SAPM_ratio_1,Str_Acc_1,Str_Def_1,TD_Acc_1,TD_Def_1,SubPM_1,PassPM_1,RevPM_1,Head_SLPM_1,Body_SLPM_1,Leg_SLPM_1,Dist_SLPM_1,Clinch_SLPM_1,Ground_SLPM_1,wins_1,losses_1,implied_prob_1,prev_1,height_1,weight_1,reach_1,age_1,stance_Orthodox_1,stance_Southpaw_1,stance_Switch_1
938,Alistair Overeem vs Roy Nelson,2015-03-14,Roy-Nelson,56.383333,991.0,806.0,22.0,23.0,0.070044,0.000000,1.704282,0.396245,1.213001,0.308447,0.005746,0.009133,0.058712,0.115217,0.000000,0.016427,0.000000,1.350594,0.227360,0.126328,1.239771,0.258435,0.206076,7,6,0.357143,13,72.0,263.0,72.0,38.0,1,0,0
417,Edson Barboza vs Kevin Lee,2018-04-21,Edson-Barboza,93.050000,1862.0,1869.0,35.0,44.0,0.019665,0.000000,1.584809,0.248992,1.493655,0.225384,0.004075,0.005624,0.117143,0.243007,0.008707,0.010538,0.000000,0.847731,0.341016,0.396063,1.407833,0.029559,0.147418,13,5,0.392157,18,71.0,155.0,75.0,32.0,1,0,0
1289,Johny Hendricks vs Stephen Thompson,2016-02-06,Johny-Hendricks,80.700000,1606.0,1341.0,53.0,80.0,0.064497,0.000000,1.579997,0.291542,1.185315,0.116347,0.004438,0.006240,0.059748,0.071179,0.008851,0.023013,0.000000,1.093823,0.307067,0.179107,0.828193,0.493205,0.258599,12,3,0.704225,15,69.0,185.0,69.0,32.0,0,1,0
615,Jim Miller vs Danny Castillo,2015-07-25,Danny-Castillo,67.466667,823.0,752.0,39.0,62.0,0.024860,0.014822,0.994368,0.306697,0.957205,0.326799,0.006241,0.008421,0.074786,0.083529,0.032468,0.048127,0.017332,0.484676,0.180873,0.328819,0.622347,0.164534,0.207487,7,5,0.476190,12,69.0,155.0,71.0,35.0,1,0,0
969,Brad Tavares vs Thales Leites,2017-10-07,Thales-Leites,101.716667,1200.0,1329.0,102.0,24.0,0.015917,0.009831,0.869445,0.456280,0.974502,0.287627,0.006183,0.007738,0.050811,0.116071,0.034083,0.089994,0.001404,0.616812,0.069419,0.183214,0.539190,0.100165,0.230090,12,6,0.384615,18,73.0,185.0,78.0,36.0,1,0,0
470,Tony Ferguson vs Gleison Tibau,2015-02-28,Gleison-Tibau,140.633333,1806.0,1763.0,156.0,69.0,0.004399,0.000000,0.730136,0.227649,0.716796,0.198496,0.004778,0.008978,0.069253,0.216069,0.023542,0.069024,0.003809,0.546610,0.116536,0.066990,0.486431,0.119350,0.124356,16,8,0.309598,24,70.0,155.0,71.0,31.0,0,1,0
1356,Yves Jabouin vs Felipe Arantes,2015-08-23,Yves-Jabouin,52.750000,812.0,662.0,17.0,25.0,0.002708,0.000000,1.078558,0.260254,0.803939,0.129657,0.004967,0.008022,0.115196,0.229524,0.009614,0.024374,0.017738,0.582313,0.308166,0.188078,0.826773,0.115296,0.136489,5,4,0.327869,9,67.0,135.0,68.0,36.0,1,0,0
533,Gray Maynard vs Nik Lentz,2018-10-06,Gray-Maynard,100.383333,1414.0,1525.0,64.0,69.0,0.088677,0.009962,1.095625,0.293181,0.662905,0.135477,0.004913,0.007102,0.093477,0.118585,0.013915,0.048386,0.004269,0.855469,0.189076,0.051080,0.590730,0.237212,0.267683,11,6,0.317460,17,69.0,155.0,70.0,39.0,1,0,0
243,Tatiana Suarez vs Carla Esparza,2018-09-08,Carla-Esparza,43.716667,657.0,590.0,35.0,47.0,0.000000,0.000000,1.153048,0.476364,0.759393,0.117923,0.004950,0.007642,0.076871,0.089056,0.006662,0.062850,0.003268,0.985711,0.131391,0.035946,0.545149,0.207917,0.399982,4,3,0.222222,7,61.0,115.0,63.0,30.0,0,0,0
646,Jose Aldo vs Max Holloway,2017-06-03,Jose-Aldo,64.200000,1279.0,1328.0,17.0,64.0,0.078478,0.000000,1.298301,0.240555,0.802383,0.103518,0.003266,0.004559,0.140523,0.093342,0.005192,0.021807,0.000000,0.990291,0.108006,0.200004,1.025714,0.030817,0.241770,8,1,0.584795,9,67.0,145.0,70.0,30.0,1,0,0


In [56]:
fighter2

,fight,date,fighter_2,cumultime_2,cumulStr_Att_fighter_2,cumulStr_Att_oponent_2,cumulTD_Att_fighter_2,cumulTD_Att_oponent_2,KDPM_2,KDPM_ratio_2,SLPM_2,SLPM_ratio_2,SAPM_2,SAPM_ratio_2,Str_Acc_2,Str_Def_2,TD_Acc_2,TD_Def_2,SubPM_2,PassPM_2,RevPM_2,Head_SLPM_2,Body_SLPM_2,Leg_SLPM_2,Dist_SLPM_2,Clinch_SLPM_2,Ground_SLPM_2,wins_2,losses_2,implied_prob_2,prev_2,height_2,weight_2,reach_2,age_2,stance_Orthodox_2,stance_Southpaw_2,stance_Switch_2
938,Alistair Overeem vs Roy Nelson,2015-03-14,Alistair-Overeem,22.516667,168.0,261.0,7.0,3.0,0.048292,0.022206,1.169837,0.464874,2.522379,1.353868,0.016247,0.006431,0.119048,0.666667,0.006345,0.006345,0.000000,0.885447,0.216821,0.067569,0.639187,0.247477,0.283172,3,3,0.649351,6,76.0,265.0,80.0,34.0,1,0,0
417,Edson Barboza vs Kevin Lee,2018-04-21,Kevin-Lee,65.833333,827.0,1016.0,44.0,45.0,0.000000,0.000000,0.883885,0.147262,1.225874,0.368893,0.006146,0.005955,0.065431,0.123272,0.025152,0.044842,0.000000,0.561697,0.222369,0.099819,0.636277,0.109170,0.138438,9,3,0.591716,12,69.0,170.0,77.0,25.0,1,0,0
1289,Johny Hendricks vs Stephen Thompson,2016-02-06,Stephen-Thompson,35.866667,535.0,524.0,13.0,19.0,0.006219,0.013941,1.156838,0.158998,1.437994,0.354350,0.005278,0.006445,0.038462,0.120108,0.000000,0.003983,0.003983,0.518839,0.272295,0.365703,0.943519,0.201370,0.011949,6,1,0.299401,7,72.0,170.0,75.0,32.0,1,0,0
615,Jim Miller vs Danny Castillo,2015-07-25,Jim-Miller,91.600000,1379.0,1013.0,63.0,29.0,0.012972,0.000000,1.129775,0.396713,0.693205,0.201412,0.004865,0.011578,0.138473,0.265517,0.064909,0.080541,0.003119,0.718582,0.208417,0.202776,0.631183,0.207402,0.291190,13,5,0.534759,18,68.0,155.0,71.0,31.0,0,1,0
969,Brad Tavares vs Thales Leites,2017-10-07,Brad-Tavares,86.366667,1174.0,1118.0,45.0,65.0,0.018461,0.011579,1.014435,0.263646,0.849236,0.146427,0.005841,0.007274,0.093704,0.148816,0.005980,0.019213,0.004962,0.576075,0.178404,0.259956,0.660712,0.251263,0.102460,10,4,0.628931,14,73.0,185.0,74.0,29.0,1,0,0
470,Tony Ferguson vs Gleison Tibau,2015-02-28,Tony-Ferguson,41.183333,771.0,646.0,6.0,22.0,0.009391,0.000000,1.130481,0.280324,0.826973,0.161306,0.003592,0.008128,0.416667,0.212879,0.030435,0.010489,0.003469,0.733841,0.190889,0.205751,1.061484,0.047953,0.021044,7,1,0.699301,8,71.0,155.0,76.0,31.0,1,0,0
1356,Yves Jabouin vs Felipe Arantes,2015-08-23,Felipe-Arantes,38.533333,517.0,489.0,13.0,25.0,0.003707,0.000000,0.826463,0.174372,0.817439,0.277643,0.004943,0.006641,0.112179,0.092000,0.007415,0.033226,0.018467,0.503922,0.237271,0.085269,0.559742,0.140880,0.125841,3,3,0.675676,6,68.0,135.0,73.0,27.0,1,0,0
533,Gray Maynard vs Nik Lentz,2018-10-06,Nik-Lentz,109.683333,1392.0,1399.0,66.0,153.0,0.005484,0.000000,1.167376,0.237968,1.143597,0.216191,0.007020,0.006076,0.106041,0.077417,0.029445,0.051237,0.004259,0.767369,0.253134,0.146873,0.637621,0.276917,0.252838,12,6,0.689655,18,68.0,155.0,68.0,34.0,1,0,0
243,Tatiana Suarez vs Carla Esparza,2018-09-08,Tatiana-Suarez,13.450000,133.0,36.0,8.0,3.0,0.000000,0.000000,1.237391,1.818702,0.212484,0.043043,0.017007,0.054516,0.225000,0.111111,0.020004,0.138078,0.000000,0.886830,0.223105,0.127456,0.136839,0.140668,0.959884,3,0,0.800000,3,65.0,115.0,66.0,27.0,0,0,0
646,Jose Aldo vs Max Holloway,2017-06-03,Max-Holloway,90.716667,2408.0,2159.0,16.0,84.0,0.009578,0.000000,1.869720,0.264903,1.446777,0.175281,0.002744,0.004715,0.234375,0.119907,0.007052,0.021835,0.000000,1.305720,0.394927,0.169073,1.441886,0.294687,0.133147,13,3,0.434783,16,71.0,145.0,69.0,25.0,1,0,0


In [57]:

w1 = [1] * (len(fighter1)//2)
w2 = [2] * (len(fighter1) - len(w1))
w = w1 + w2

In [58]:

len(w)

1454

In [59]:
fighter1['winner'] = w
fighter2['winner'] = w

In [60]:
fighter1.to_csv('fighter1.csv')
fighter2.to_csv('fighter2.csv')

In [61]:
len(fighter1)

1454

In [62]:
len(fighter2)

1454

In [85]:
real_model_df = pd.merge(fighter1, fighter2, how='outer')
real_model_df

,fight,date,fighter_1,cumultime_1,cumulStr_Att_fighter_1,cumulStr_Att_oponent_1,cumulTD_Att_fighter_1,cumulTD_Att_oponent_1,KDPM_1,KDPM_ratio_1,SLPM_1,SLPM_ratio_1,SAPM_1,SAPM_ratio_1,Str_Acc_1,Str_Def_1,TD_Acc_1,TD_Def_1,SubPM_1,PassPM_1,RevPM_1,Head_SLPM_1,Body_SLPM_1,Leg_SLPM_1,Dist_SLPM_1,Clinch_SLPM_1,Ground_SLPM_1,wins_1,losses_1,implied_prob_1,prev_1,height_1,weight_1,reach_1,age_1,stance_Orthodox_1,stance_Southpaw_1,stance_Switch_1,winner,fighter_2,cumultime_2,cumulStr_Att_fighter_2,cumulStr_Att_oponent_2,cumulTD_Att_fighter_2,cumulTD_Att_oponent_2,KDPM_2,KDPM_ratio_2,SLPM_2,SLPM_ratio_2,SAPM_2,SAPM_ratio_2,Str_Acc_2,Str_Def_2,TD_Acc_2,TD_Def_2,SubPM_2,PassPM_2,RevPM_2,Head_SLPM_2,Body_SLPM_2,Leg_SLPM_2,Dist_SLPM_2,Clinch_SLPM_2,Ground_SLPM_2,wins_2,losses_2,implied_prob_2,prev_2,height_2,weight_2,reach_2,age_2,stance_Orthodox_2,stance_Southpaw_2,stance_Switch_2
0,Alistair Overeem vs Roy Nelson,2015-03-14,Roy-Nelson,56.383333,991.0,806.0,22.0,23.0,0.070044,0.000000,1.704282,0.396245,1.213001,0.308447,0.005746,0.009133,0.058712,0.115217,0.000000,0.016427,0.000000,1.350594,0.227360,0.126328,1.239771,0.258435,0.206076,7,6,0.357143,13,72.0,263.0,72.0,38.0,1,0,0,1,Alistair-Overeem,22.516667,168.0,261.0,7.0,3.0,0.048292,0.022206,1.169837,0.464874,2.522379,1.353868,0.016247,0.006431,0.119048,0.666667,0.006345,0.006345,0.000000,0.885447,0.216821,0.067569,0.639187,0.247477,0.283172,3,3,0.649351,6,76.0,265.0,80.0,34.0,1,0,0
1,Edson Barboza vs Kevin Lee,2018-04-21,Edson-Barboza,93.050000,1862.0,1869.0,35.0,44.0,0.019665,0.000000,1.584809,0.248992,1.493655,0.225384,0.004075,0.005624,0.117143,0.243007,0.008707,0.010538,0.000000,0.847731,0.341016,0.396063,1.407833,0.029559,0.147418,13,5,0.392157,18,71.0,155.0,75.0,32.0,1,0,0,1,Kevin-Lee,65.833333,827.0,1016.0,44.0,45.0,0.000000,0.000000,0.883885,0.147262,1.225874,0.368893,0.006146,0.005955,0.065431,0.123272,0.025152,0.044842,0.000000,0.561697,0.222369,0.099819,0.636277,0.109170,0.138438,9,3,0.591716,12,69.0,170.0,77.0,25.0,1,0,0
2,Johny Hendricks vs Stephen Thompson,2016-02-06,Johny-Hendricks,80.700000,1606.0,1341.0,53.0,80.0,0.064497,0.000000,1.579997,0.291542,1.185315,0.116347,0.004438,0.006240,0.059748,0.071179,0.008851,0.023013,0.000000,1.093823,0.307067,0.179107,0.828193,0.493205,0.258599,12,3,0.704225,15,69.0,185.0,69.0,32.0,0,1,0,1,Stephen-Thompson,35.866667,535.0,524.0,13.0,19.0,0.006219,0.013941,1.156838,0.158998,1.437994,0.354350,0.005278,0.006445,0.038462,0.120108,0.000000,0.003983,0.003983,0.518839,0.272295,0.365703,0.943519,0.201370,0.011949,6,1,0.299401,7,72.0,170.0,75.0,32.0,1,0,0
3,Jim Miller vs Danny Castillo,2015-07-25,Danny-Castillo,67.466667,823.0,752.0,39.0,62.0,0.024860,0.014822,0.994368,0.306697,0.957205,0.326799,0.006241,0.008421,0.074786,0.083529,0.032468,0.048127,0.017332,0.484676,0.180873,0.328819,0.622347,0.164534,0.207487,7,5,0.476190,12,69.0,155.0,71.0,35.0,1,0,0,1,Jim-Miller,91.600000,1379.0,1013.0,63.0,29.0,0.012972,0.000000,1.129775,0.396713,0.693205,0.201412,0.004865,0.011578,0.138473,0.265517,0.064909,0.080541,0.003119,0.718582,0.208417,0.202776,0.631183,0.207402,0.291190,13,5,0.534759,18,68.0,155.0,71.0,31.0,0,1,0
4,Brad Tavares vs Thales Leites,2017-10-07,Thales-Leites,101.716667,1200.0,1329.0,102.0,24.0,0.015917,0.009831,0.869445,0.456280,0.974502,0.287627,0.006183,0.007738,0.050811,0.116071,0.034083,0.089994,0.001404,0.616812,0.069419,0.183214,0.539190,0.100165,0.230090,12,6,0.384615,18,73.0,185.0,78.0,36.0,1,0,0,1,Brad-Tavares,86.366667,1174.0,1118.0,45.0,65.0,0.018461,0.011579,1.014435,0.263646,0.849236,0.146427,0.005841,0.007274,0.093704,0.148816,0.005980,0.019213,0.004962,0.576075,0.178404,0.259956,0.660712,0.251263,0.102460,10,4,0.628931,14,73.0,185.0,74.0,29.0,1,0,0
5,Tony Ferguson vs Gleison Tibau,2015-02-28,Gleison-Tibau,140.633333,1806.0,1763.0,156.0,69.0,0.004399,0.000000,0.730136,0.227649,0.716796,0.198496,0.004778,0.008978,0.069253,0.216069,0.023542,0.069024,0.003809,0.546610,0.116536,0.066990,0.486431,0.119350,0.124356,16,8,0.30959